In [1]:
#using Parameters       # @unpack
#using NBInclude        # including notebooks instead of jl files
#using SparseArrays     # SparseMatrixCSC

#@nbinclude("Structures_v2.ipynb");


function Multi(proj::Projection,solution::AiyagariSolution,params::Params,μ)

    @unpack N,Ntot,ind_h,allocation_proj,ξs = proj
    @unpack S_h,Π_h,y0_h,a_beg_h,a_end_h,c_h,l_h,ly_h,u_h,u′_h,u′′_h,v_h,v′_h,nb_cc_h,ind_cc_h = (
                allocation_proj)
    @unpack ξu0,ξu1,ξu2,ξuE,ξy,ξv0,ξv1 = ξs 
    @unpack β,α,ϵ,κ,δ,tk,Tt,u′,u′′,v′,v′′,ys = params
    @unpack w,R,K,L,G,Y,A = solution

    T = typeof(w)
    r = R - 1


    ξv0_tilde = ξv0./(y0_h)
    ξv1_tilde = ξv1./(y0_h)


    ω = ones(T,1,Ntot)'

    # S_h entry non null  

    # Π_h is transposed compared to the older version of the code 

    Π_h_ψ = spdiagm(S_h)*Π_h*spdiagm(one(T)./S_h) #### verify ∀x :  Π_h_ψ*(S_h.*x)=S_h.(Π_h*x)  
    Is = sparse(I,Ntot,Ntot)
    P  = Is - sparse(ind_cc_h,ind_cc_h,ones(T,nb_cc_h),Ntot,Ntot)   



    M0 = w*L*Is - (α-1)*(r+(1-tk)*δ)*S_h*a_beg_h'- α*w*S_h*(y0_h.*l_h)' 

    L1t = Is - β*(1+r)*Π_h_ψ - (β/K)*(α-1)*(r+(1-tk)*δ)*S_h*a_beg_h'- (β/K)*α*w*S_h*(y0_h.*l_h)' 

   # Attention L1t* ω neq L1t
   # XR3 : here as well
    x1 = L1t* (S_h .* ξu0 .* u′.(c_h)) - (β/K)*μ*(α*Y - (α-1)*(r+ (1-tk)*δ)*A - (r+δ)*K - α*w*L)*S_h  

   # XR4 : alpha missing, (1-tk) missing
    L1l = L1t*spdiagm(w*y0_h.*ξu1.*u′′.(c_h)) -  (β/K)*α*w*S_h*(y0_h.*ξu1.*u′.(c_h))'*Π_h' 
    # XR5 :  (1-tk) missing
    L1c = L1t*spdiagm(ξuE.*u′′.(c_h))*(Is-(1+r)*Π_h') + (β/K)*(α-1)*(r+(1-tk)*δ)*S_h*(ξuE.*u′.(c_h))'*Π_h' 


    L0t = M0*spdiagm(ξuE.*u′′.(c_h))*(Is-(1+r)*Π_h')+ (α-1)*(r+ (1-tk)*δ)*S_h*(ξuE.*u′.(c_h))'*Π_h' #
    # L0 = L0t\(M0*spdiagm(w*y0_h.*ξu1.*u′′.(c_h)) - spdiagm(L* ξv1_tilde.*v′′.(l_h)) - α*w*(S_h*(y0_h.*ξu1.*u′.(c_h))'*Π_h')) #
    

    # Correction erreur lambdal et non lambdal_tilde : I remove the Π_h' after α*w*(S_h*(y0_h.*ξu1.*u′.(c_h))' 
    L0 = L0t\(M0*spdiagm(w*y0_h.*ξu1.*u′′.(c_h)) - spdiagm(L* ξv1_tilde.*v′′.(l_h)) - α*w*(S_h*(y0_h.*ξu1.*u′.(c_h))')) #

    
    # x0 =  L0t\(M0*S_h.*ξu0.*u′.(c_h) - L*S_h.*ξv0_tilde.*v′.(l_h) + μ*(1-α)*(Y - (r+ (1-tk)*δ)*A - w*L)*S_h) #

    ###### !!!!!!!!!!! XR1 : paranthesis was missing !!!!!
    x0 =  L0t\(M0*(ω.*S_h.*ξu0.*u′.(c_h)) - L*S_h.*ξv0_tilde.*v′.(l_h) + μ*(1-α)*(Y - (r+ (1-tk)*δ)*A - w*L)*S_h) 


    # L2 = inv(I - P + P*L1c)*P*L1l  
    # x2 = inv(I - P + P*L1c)*P*x1
    

    L2 = (I - P + P*L1c)\(P*L1l) 
    x2 = (I - P + P*L1c)\(P*x1)


    # lambdal = - spdiagm(one(T) ./S_h)*(one(T)./(L0 - L2))*(x0-x2) 
    # lambdac = spdiagm(one(T) ./S_h)*(x2 - L2*(inv(L2 - L0)*(x2-x0) ))
    

    # by bin lambda_bar
    lambdal_b = - (L0 - L2)\(x0-x2) # end-of-period
    lambdac_b = x2 - L2*inv(L2 - L0)*(x2-x0) 
    lambdac_t_b = Π_h'*lambdac_b # beginning of period
    lambdal_t_b = Π_h'*lambdal_b # beginning of period

    # S_h.*ψ = S_h.*ξu0.*u′.(c_h)) - spdiagm(ξuE.*u′′.(c_h))*(Is - (1+r)*Π')*x2 + spdiagm(w*y0_h.*ξu1.*u′′.(c_h))*(S_h.*lambdal) + 
    # spdiagm(ξuE.*u′′.(c_h))*(Is - (1+r)*Π') - spdiagm(w*y0_h.*ξu1.*u′′.(c_h))*inv(L0 - L2)*(x0-x2)

  #  ψ = spdiagm(one(T)./S_h)*(S_h.*ξu0.*u′.(c_h) - spdiagm(ξuE.*u′′.(c_h))*(Is - (1+r)*Π_h')*x2 + spdiagm(w*y0_h.*ξu1.*u′′.(c_h))*(S_h.*lambdal) + 
  #   spdiagm(ξuE.*u′′.(c_h))*(Is - (1+r)*Π_h') - spdiagm(w*y0_h.*ξu1.*u′′.(c_h))*inv(L0 - L2)*(x0-x2))

# by bin 
    ψ_b =(S_h.*ξu0.*u′.(c_h) - spdiagm(ξuE.*u′′.(c_h))*(Is - (1+r)*Π_h')*x2 + 
    spdiagm(ξuE.*u′′.(c_h))*(Is - (1+r)*Π_h')*L2*inv(L2 - L0)*(x2-x0) - spdiagm(w*y0_h.*ξu1.*u′′.(c_h))*inv(L0 - L2)*(x0-x2))
   
    # Checking the equations

#   CC = zeros(9)


#   CC[1] = maximum(abs.(-ψ_b + S_h.*ξu0.*u′.(c_h) - spdiagm(ξuE.*u′′.(c_h))*(Is - (1+r)*Π_h')*lambdac_b +  spdiagm(w*y0_h.*ξu1.*u′′.(c_h))*lambdal_b))
   
#   Sc1 =  ( a_beg_h'*ψ_b  +(ξuE.*u′.(c_h))'*lambdac_t_b)[1]
#   Sc2 = ( (y0_h.*l_h)'*ψ_b + (y0_h.*ξu1.*u′.(c_h))'*lambdal_t_b )[1]

#   CC[2] = maximum(abs.(-w*L*ψ_b +
#       L*S_h.*ξv0_tilde.*v′.(l_h) + L*ξv1_tilde.*v′′.(l_h).* lambdal_b + 
#     (α-1)*(r+(1-tk)*δ)*Sc1*S_h +
#       α*w*Sc2*S_h -
#       μ*(1-α)*(Y - (r+ (1-tk)*δ)*A - w*L)*S_h))  

#       # Double check
#  CC[3] = maximum(abs.(  - w*L*ψ_b + 
#       L*S_h.*ξv0_tilde.*v′.(l_h) + spdiagm(L* ξv1_tilde.*v′′.(l_h))*lambdal_b+
#       (α-1)*(r+ (1-tk)*δ)*S_h*( a_beg_h'*ψ_b  +(ξuE.*u′.(c_h))'*lambdac_t_b )+
#       + α*w*S_h*( (y0_h.*l_h)'*ψ_b + (y0_h.*ξu1.*u′.(c_h))'*lambdal_t_b ) - μ*(1-α)*(Y - (r+ (1-tk)*δ)*A - w*L)*S_h 
#        ))

 
   

#  CC[4] =  maximum(abs.( -P*ψ_b +
#         β*(1+r)*P*Π_h_ψ*ψ_b +
#       (β/K)*(α-1)*(r+ (1-tk)*δ)*P*( S_h*a_beg_h'*ψ_b  +S_h*(ξuE.*u′.(c_h))'*lambdac_t_b )+
#       (β/K)*α*w*P*( S_h*(y0_h.*l_h)'*ψ_b + S_h*(y0_h.*ξu1.*u′.(c_h))'*lambdal_t_b )+
#       (β/K)*μ*(α*Y - (α-1)*(r+δ)*A -(r+δ)*K - α*w*L)*P*S_h 
#     ))

# #CC[5] = maximum(abs.(   P*L1t*(  S_h .* ξu0 .* u′.(c_h) - spdiagm(ξuE.*u′′.(c_h))*(Is-(1+r)*Π_h')*lambdac_b + spdiagm(w*y0_h.*ξu1.*u′′.(c_h))*lambdal_b   )

# CC[5] = maximum(abs.(  - P*L1t*ψ_b
#  + P*(β/K)*(α-1)*(r+(1-tk)*δ)*S_h*(ξuE.*u′.(c_h))'*Π_h'*lambdac_b + 
#  +P*(β/K)*α*w*S_h*(y0_h.*ξu1.*u′.(c_h))'*Π_h'*lambdal_b +
#  + P*(β/K)*μ*(α*Y - (α-1)*(r+ (1-tk)*δ)*A - (r+δ)*K - α*w*L)*S_h  
#     ))


#     # CC[6] = maximum(abs.(S_h.*ξuE.*u′.(c_h) -β*(1+r)*Π_h*(S_h.*ξuE.*u′.(c_h))  )  )

    # Resid = test(2,solution,params);
    # CC[6] = maximum(abs.(ξuE.*u′.(c_h) -β*(1+r)*Π_h*(ξuE.*u′.(c_h))  .- Resid))


    # CC[7] = maximum(abs.(S_h.*ψ_b - 1/w*S_h.*ω.*ξv0.*v′.(l_h) + 1/w*S_h.*ξv1.*v′′.(l_h).*(S_h.*lambdal_b) + μ/(w*L)*(1-α)*(Y - (r + (1-tk)*δ)*A -w*L)*S_h
    #                   - α/L*(S_h*(y0_h.*l_h)'*(S_h.*ψ_b) + S_h*(y0_h.*ξu1.*u′.(c_h))'*(S_h.*lambdal_t_b)   )  )) 




# deriv a


# deriv l
# υ = Upsilon, si en majuscule, ressemble exactement à Y 
  # CC[7] = maximum(abs.(ψ_b - (1/w)*S_h.*ω.*ξv0_tilde.*v′.(l_h) - (1/w)*ξv1_tilde.*v′′.(l_h).*lambdal_b 
  #                         - α/L*(S_h*(y0_h.*l_h)'*(ψ_b) + S_h*(y0_h.*ξu1.*u′.(c_h))'*(lambdal_t_b)) 
  #                          - 1/(w*L)*(1-α)* (μ*(Y - w*L) - (r/(1-tk) + δ)*(β^(-1)*υ ) ) S_h
  #                          )) 
  # CC[7] = ψ_b

# deriv r


  # CC[8] = maximum(abs.( 0 .- S_h*a_beg_h'*ψ_b .- S_h*(ξuE.*u′.(c_h))'*lambdac_t_b .+ μ*A ))
  # CC[8] = maximum(abs.( 0 .- a_beg_h'*ψ_b - (ξuE.*u′.(c_h))'*lambdac_t_b .+ μ*A ))

# deriv zeta 


# epss = 6
# ζ = 1



# # CC[8] = maximum(abs.( 0 .- a_beg_h'*ψ_b - (ξuE.*u′.(c_h))'*lambdac_t_b .+ μ*A ))
# CC[8] = maximum(abs.( 0  ))

# # w*μ*L - (epss-1)/κ*ζ*Y  =  spdiagm(w*y0_h.*ξu1.*u′.(c_h))*lambdal_b
               
#   # link CC[4]
#   CC[9] = maximum(abs.( P*ψ_b - β*(1+r)*P*Π_h_ψ*ψ_b 
#                               -  (β/K)*α*w*P*( S_h*(y0_h.*l_h)'*ψ_b + S_h*(y0_h.*ξu1.*u′.(c_h))'*lambdal_t_b )
#                               - (β/K)*α*P*μ*(α*Y -(r+δ)*K - α*w*L)*S_h 
#                               + (β/K)*(α-1)*(r/(1-tk) +δ)*(β^(-1)*υ + Γ*(1-tk)*K)
#                               ))


#                               CC[4] =  maximum(abs.( -P*ψ_b +
#                             β*(1+r)*P*Π_h_ψ*ψ_b +
#                             (β/K)*α*w*P*( S_h*(y0_h.*l_h)'*ψ_b + S_h*(y0_h.*ξu1.*u′.(c_h))'*lambdal_t_b )+
#                             (β/K)*α*μ*(α*Y - (α-1)*(r+δ)*A -(r+δ)*K - α*w*L)*P*S_h 
#                           ))
                      
                      

                              

    # CC[5] = maximum(abs.(ψ_b - S_h* ))

    #resid = μ .-  ones(T,1,Ntot)*(S_h.*ψ)

    ###### XR2 : Ici
    resid = μ .-  ones(T,1,Ntot)*(ψ_b)
    

return resid, lambdal_b, lambdac_b, ψ_b
# ,CC,(S_h.*ξu0.*u′.(c_h))

end


LoadError: LoadError: UndefVarError: `@unpack` not defined
in expression starting at /Users/am13232/Dropbox/optimal_money_New/Code_Wage/AMB/Sticky_Prices/steady_state/Ramsey_SP.ipynb:10

In [39]:

function multi_find(proj::Projection,solution::AiyagariSolution,params::Params)
   
    @unpack allocation_proj,ξs = proj
    @unpack S_h, Π_h,a_beg_h,c_h,u′_h,l_h,y0_h = (allocation_proj)            
    @unpack A,B,L,Y,R,w,K = solution
    @unpack β,tk,u′,κ,δ,ϵ,ν,ys,Sy = params
    @unpack ξuE,ξu1 = ξs 
    # @unpack ξu0,ξu1,ξu2,ξuE,ξy,ξv0,ξv1 = ξs 

    r = R - 1

    μ_init =   100
    fw(μ) = Multi(proj,solution,params,μ)[1]
    μ_solut     = find_zero(fw, μ_init)  

    lambdal = Multi(proj,solution,params,μ_solut)[2] 
    lambdac =  Multi(proj,solution,params,μ_solut)[3] 
    ψ_b  =  Multi(proj,solution,params,μ_solut)[4] 

    lambdact = Π_h'*lambdac
    lambdalt = Π_h'*lambdal

    # lambdact = spdiagm(one(T)./S_h)*Π_h'*(S_h.*lambdac) 
    # lambdalt = spdiagm(one(T)./S_h)*Π_h'*(S_h.*lambdal) 

    #epss = 6 # mettre ça dans les params

    Γ = (1/A)*(  μ_solut*A .- a_beg_h'*ψ_b - (ξuE.*u′.(c_h))'*lambdact) 
    
    υ  =  β*(1-tk)*Γ*B 
    Lγ = κ./(Y*(ϵ-1)).*( μ_solut*w*L .+ (r/(1-tk)+δ)*(β^(-1)*υ + Γ*(1-tk)*K)  .-  
        w*((y0_h.*l_h)'*ψ_b .+ (y0_h.*ξu1.*u′.(c_h))'*lambdal) .- μ_solut*Y .+ Y*sum((y0_h.^ν).*ψ_b)/sum((y0_h.^ν).*S_h))
    
    @show sum((y0_h.^ν).*S_h) - sum((ys.^ν).*Sy)

    plan = Plan(
        vec(lambdac),
        vec(lambdact),
        vec(lambdal),
        vec(lambdalt),
        vec(ψ_b),
        μ_solut,
        Γ[1],
        υ[1],
        Lγ[1]
        )


    return plan

end   
    



multi_find (generic function with 1 method)